In [363]:
import pandas as pd

In [393]:
# Read the CSV files into DataFrames
july_2023 = pd.read_csv("July-01-to-2023-July-31.csv", parse_dates=True)
full_year_df = pd.read_csv("full_year-may23-may24.csv", parse_dates=True)
may_july_2024 = pd.read_csv("may-16-to-July-07-2024.csv", parse_dates=True)

In [394]:
july_2023.rename(columns = {'name':'location'}, inplace = True)
full_year_df.rename(columns = {'name':'location'}, inplace = True)
may_july_2024.rename(columns = {'name':'location'}, inplace = True)

In [395]:
print(july_2023["location"].value_counts())
print(full_year_df["location"].value_counts())
print(may_july_2024["location"].value_counts())

location
Ottowa           31
London           31
Washington DC    31
Beijing          31
Moscow           31
Berlin           31
Paris            31
Rome             31
Mexico City      31
Name: count, dtype: int64
location
Beijing          381
Berlin           381
London           381
Mexico City      381
Ottawa           381
Washington DC    381
Moscow           289
Rome             197
Paris            184
Rome             184
Paris            105
Moscow            92
Paris             92
Name: count, dtype: int64
location
London           53
Washington DC    53
Beijing          53
Moscow           53
Ottawa           53
Berlin           53
Paris            53
Rome             53
Mexico City      53
Name: count, dtype: int64


# May 16th 2024 - July 7th 2024

In [398]:
may_july_2024.tail(1)

,location,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
476,Mexico City,2024-07-07,24.1,13.6,17.6,24.1,13.6,17.6,10.7,66.6,...,23.3,10,10,2024-07-07T06:04:11,2024-07-07T19:19:02,0.06,"Rain, Partially cloudy",Partly cloudy throughout the day with rain.,rain,NaN


# July 1st 2023 - May 15th 2024
Join tables

In [391]:
full_year_df.tail(1)
july_2023.head(1)

,location,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,Ottowa,2023-07-01,27.0,19.6,23.1,28.6,19.6,23.2,20.8,88.0,...,12.5,7.0,30.0,2023-07-01T05:17:46,2023-07-01T20:26:19,0.44,"Rain, Partially cloudy",Partly cloudy throughout the day with morning ...,rain,"72212604879,KVPZ,KPPO,72102900347,72532704846,..."


In [368]:
correct_dtypes = {
    'location': 'object',
    'tempmax': 'float64',
    'tempmin': 'float64',
    'temp': 'float64',
    'feelslikemax': 'float64',
    'feelslikemin': 'float64',
    'feelslike': 'float64',
    'dew': 'float64',
    'humidity': 'float64',
    'precip': 'float64',
    'precipprob': 'int64',
    'precipcover': 'float64',
    'preciptype': 'object',
    'snow': 'float64',
    'snowdepth': 'float64',
    'windgust': 'float64',
    'windspeed': 'float64',
    'winddir': 'float64',
    'sealevelpressure': 'float64',
    'cloudcover': 'float64',
    'visibility': 'float64',
    'solarradiation': 'float64',
    'solarenergy': 'float64',
    'uvindex': 'float64',
    'severerisk': 'float64',
    'sunrise': 'object',
    'sunset': 'object',
    'moonphase': 'float64',
    'conditions': 'object',
    'description': 'object',
    'icon': 'object',
    'stations': 'object'
}
july_2023 = july_2023.astype(correct_dtypes)

In [387]:
# Concatenate the dataframes along the index
df = pd.merge(full_year_df, july_2023, how="outer").set_index("datetime")

df.tail()
df[df["location"] == "Ottawa"].value_counts().sum()
df[df["location"] == "Mexico City"].value_counts().sum()


location
Beijing          381
Berlin           381
London           381
Mexico City      381
Ottawa           381
Washington DC    381
Moscow           289
Rome             197
Paris            184
Rome             184
Paris            105
Moscow            92
Paris             92
Ottowa            31
Name: count, dtype: int64

Merge of the two csv datasets plus a new one, expanding from November 1st 2023 to August 1st 2023 - May 15 2024
Source: https://www.visualcrossing.com/weather/weather-data-services#

In [389]:
# city values remove extra spaces
df['location'] = df['location'].str.strip()
# calculate null value percentage in columns
null_pct = df.apply(pd.isnull).sum()/df.shape[0]
# use valid_columns as index of columns to keep which have less than 5% null values
valid_columns = df.columns[null_pct < .05]
df = df[valid_columns].copy()

In [390]:
df["location"].value_counts()

location
Beijing          381
Berlin           381
London           381
Mexico City      381
Moscow           381
Ottawa           381
Paris            381
Rome             381
Washington DC    381
Ottowa            31
Name: count, dtype: int64

In [371]:
df = df[['location', 'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility',
       'moonphase', 'conditions', 'description']]

# df = df[['location', 'dew', 'humidity', 'precip', 'precipcover', 'windgust', 'cloudcover', 'visibility']]

In [379]:
beijing_city = df[df["location"] == "Beijing"]
berlin_city = df[df["location"] == "Berlin"]
London_city = df[df["location"] == "London"]
mexico_city = df[df["location"] == "Mexico City"]
moscow_city = df[df["location"] == "Moscow"]
ottawa_city = df[df["location"] == "Ottawa"]
paris_city = df[df["location"] == "Paris"]
rome_city = df[df["location"] == "Rome"]
washington_city = df[df["location"] == "Washington DC"]

The workflow:
1) Understand the problem -- find next day's high/low temp; I have data on humidity, temperature, wind speed, etc
2) Identify Key Factors -- What are the independent variables (hint #1) and my dependent variables
3) Hypothesis -- _ significantly impacts 
4) Translate Hypo => Testable Statements -- Null/Alternate Hypothesis (this is an example, I had to do this in school, maybe it will help maybe not)
5) Design the Experiment -- Data collection/cleaning, model selection, evaluation metrics (Research here)
6) Analsys -- train the models, evaluate results (expected temp from your model vs actual temp), testing
7) Success 

- What determines the next day's high and low temperature for each location?
- What do you see from the data at first glance that might impact this?
- How would you need to organize and clean your data?
- Is your model 'universal', meaning does it apply to every location the same (no variance) when it comes to these predictions?
- If not, what factors impact your model?
- What tools or metrics will you need to use to account for these changes?
- Is your model accurate? What causes the inaccuracy, if any?

4) Translate Hypo => Testable Statements -- Null/Alternate Hypothesis (this is an example, I had to do this in school, maybe it will help maybe not)

In [373]:
df_numerical = df[['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin',
       'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover',
       'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
       'sealevelpressure', 'cloudcover', 'visibility',
       'moonphase']]

5) Design the Experiment -- Data collection/cleaning, model selection, evaluation metrics (Research here)

- I used a heatmap to visualize the corralatioin between dew point and temperature

- Check Regression assumptions
- Seperate dataframes per location
- R2 and Adjusted R2
- - sns.heatmap Pearson correlation test
- Feature selection
- Hypothesis -- significantly impacts
- Cross validation (Model selection)
- Training and evaluation

The true purpose of regression is to seperate the total variance in lowtemp and hightemp from the variance explained by dew and the variance that is still unexplained.

In [374]:
# VIF < 5: Low multicollinearity.
# VIF between 5 and 10: Moderate multicollinearity. Investigate further.
# VIF > 10: High multicollinearity. Consider removing or combining features.

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

features = ['dew', 'humidity', 'precip', 'precipcover', 'windgust', 'cloudcover', 'visibility']

# Add a constant to the model (intercept)
X = add_constant(df_numerical[features])

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

       feature        VIF
0        const  39.777236
1          dew   1.116597
2     humidity   1.830842
3       precip   1.385957
4  precipcover   1.974844
5     windgust   1.145459
6   cloudcover   1.608647
7   visibility   1.026734


- Positive coefficients for dew, precip, precipcover suggest these factors increase tempmax.
- Negative coefficients for humidity, windgust, cloudcover, visibility suggest these factors decrease tempmax.

# Hyperparameter tuning to optimize Ridge

In [375]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, make_scorer

features = ['dew', 'humidity', 'precip', 'precipcover', 'windgust', 'cloudcover', 'visibility']
target = 'tempmax'

# Assuming df_numerical is your dataframe
X = df_numerical[features]
y = df_numerical[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a scoring function
scoring = make_scorer(mean_squared_error, greater_is_better=False)

# Define the Ridge model and the parameter grid for alpha
ridge = Ridge()
param_grid = {
    'ridge__alpha': [0.01, 0.1, 1, 10, 100, 1000]
}

# Set up the pipeline with a standard scaler and ridge regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Optional: use if feature scaling is needed
    ('ridge', ridge)
])

# Initialize GridSearchCV with the pipeline and parameter grid
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring=scoring)

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation MSE: ", -grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print("Test MSE: ", test_mse)

Best parameters found:  {'ridge__alpha': 0.01}
Best cross-validation MSE:  3.7285925900551127
Test MSE:  4.530962346408037


In [376]:
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter distribution for alpha
param_distributions = {
    'ridge__alpha': [0.01, 0.1, 1, 10, 100, 1000]
}

# Initialize RandomizedSearchCV with the pipeline and parameter distributions
random_search = RandomizedSearchCV(pipeline, param_distributions, n_iter=10, cv=5, scoring=scoring, random_state=42)

# Fit RandomizedSearchCV to the training data
random_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best parameters found: ", random_search.best_params_)
print("Best cross-validation MSE: ", -random_search.best_score_)

# Evaluate the best model on the test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_pred)
print("Test MSE: ", test_mse)


/home/ahzof/hw_ds/.venv/lib/python3.10/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters found:  {'ridge__alpha': 0.01}
Best cross-validation MSE:  3.7285925900551127
Test MSE:  4.530962346408037
